# This ipynb generates MLP-like QBAFs

In [1]:
import random
random.seed(1)
import sys
sys.path.append("../../src/")
import uncertainpy.gradual as grad
import numpy as np
from tqdm import tqdm
import time

In [2]:
def generate_random_mlp_graph(layer_sizes, connection_prob):
    """
    generate a random MLP-like QBAF structure, represented by DAG

    parameter:
    - connection_prob: The probability of inter-layer connections (1.0 = fully connected, 0.0 = no connections)

    return:
    - graph: MLP structure in adjacent list (DAG)
    """

    graph = {}  # Store the DAG using an adjacency list
    node_id = 0  # neuron ID
    layer_nodes = []  # Record the neuron IDs in each layer

    # Create neuron nodes
    for size in layer_sizes:
        layer = [node_id + i for i in range(size)]
        layer_nodes.append(layer)
        node_id += size

    # Generate inter-layer connections
    for i in range(len(layer_nodes) - 1):  # Layer-by-layer connection
        for src in layer_nodes[i]:  # Current layer neurons
            for dst in layer_nodes[i+1]:  # Next layer neurons
                if random.uniform(0, 1) < connection_prob:
                    graph.setdefault(src, set()).add(dst)

    # if node not in graph, add empty set
    for layer in layer_nodes:
        for node in layer:
            graph.setdefault(node, set())

    return graph

In [3]:
def get_layer_nodes(layer_sizes, layer_index=None):
    node_id = 0
    for i, size in enumerate(layer_sizes):
        if i == layer_index:
            return [str(n) for n in range(node_id, node_id + size)]
        node_id += size

    raise ValueError(f"Invalid layer_index {layer_index}: must be between 0 and {len(layer_sizes) - 1}")

# preferred_order = get_layer_nodes(layer_sizes, len(layer_sizes)-1)

In [4]:
# generate a random MLP-QBAF and output to a file
def generate_and_write_graph(filename, layer_sizes, connection_prob):
    with open(filename, 'w') as f:
        sys.stdout = f

        # generate the node and edge of a graph
        random_graph = generate_random_mlp_graph(layer_sizes, connection_prob)

        L0 = get_layer_nodes(layer_sizes, len(layer_sizes)-1)
        L1 = get_layer_nodes(layer_sizes, len(layer_sizes)-2)
        L2 = get_layer_nodes(layer_sizes, len(layer_sizes)-3)
        L3 = get_layer_nodes(layer_sizes, len(layer_sizes)-4)
        if len(layer_sizes)>=5:
            L4 = get_layer_nodes(layer_sizes, len(layer_sizes)-5)
            L4 = list(map(int, L4))
        L0 = list(map(int, L0))
        L1 = list(map(int, L1))
        L2 = list(map(int, L2))
        L3 = list(map(int, L3))


        # generate random base scores for arguments
        for node, edges in random_graph.items():
            # print(node)
            if node in L1:
                random_float = round(random.uniform(0.0, 0.1),2)
            else:
                random_float = round(random.uniform(0.0, 1.0),2)
            print(f"arg({node}, {random_float}).")

        # generate random polarity for edges
        for node, edges in random_graph.items():
            for edge in edges:

                if edge in L1:
                    random_boolean = True
                elif edge in L2:
                    random_boolean = False
                else:
                    random_boolean = random.choice([True, False])
                if random_boolean:
                    print(f"att({node}, {edge}).")
                else:
                    print(f"sup({node}, {edge}).")

    sys.stdout = sys.__stdout__

In [5]:
# set generation parameters
connection_prob=1.0
layer_sizes = [8,8,8,3]
mlp_graph = generate_random_mlp_graph(layer_sizes, connection_prob)

print("MLP structure:", layer_sizes)

MLP structure: [8, 8, 8, 3]


In [6]:
mlp_graph

{0: {8, 9, 10, 11, 12, 13, 14, 15},
 1: {8, 9, 10, 11, 12, 13, 14, 15},
 2: {8, 9, 10, 11, 12, 13, 14, 15},
 3: {8, 9, 10, 11, 12, 13, 14, 15},
 4: {8, 9, 10, 11, 12, 13, 14, 15},
 5: {8, 9, 10, 11, 12, 13, 14, 15},
 6: {8, 9, 10, 11, 12, 13, 14, 15},
 7: {8, 9, 10, 11, 12, 13, 14, 15},
 8: {16, 17, 18, 19, 20, 21, 22, 23},
 9: {16, 17, 18, 19, 20, 21, 22, 23},
 10: {16, 17, 18, 19, 20, 21, 22, 23},
 11: {16, 17, 18, 19, 20, 21, 22, 23},
 12: {16, 17, 18, 19, 20, 21, 22, 23},
 13: {16, 17, 18, 19, 20, 21, 22, 23},
 14: {16, 17, 18, 19, 20, 21, 22, 23},
 15: {16, 17, 18, 19, 20, 21, 22, 23},
 16: {24, 25, 26},
 17: {24, 25, 26},
 18: {24, 25, 26},
 19: {24, 25, 26},
 20: {24, 25, 26},
 21: {24, 25, 26},
 22: {24, 25, 26},
 23: {24, 25, 26},
 24: set(),
 25: set(),
 26: set()}

In [7]:
N = 100 # generate N QBAFs storing in N files
for i in range(N):
    filename = f'../../bags/mlp_{i}.bag'
    generate_and_write_graph(filename, layer_sizes, connection_prob)